In [1]:
import pandas as pd
import os
import csv
from api_keys import g_key
import requests
import json

In [2]:
csvpath = os.path.join('..', 'Resources', 'cities_and_weather.csv')
with open(csvpath) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        cities_weather = pd.DataFrame(csvreader)

In [3]:
cities_weather = cities_weather.drop(cities_weather[[0]], axis=1)

In [4]:
cities_weather.columns = ["Cities","Country","Latitude","Longitude","Temperature (F)",
                                                       "Humidity (%)", "Cloudiness (%)", "Wind Speed (mph)"]

In [5]:
cities_weather.head()

,Cities,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,puerto ayora,ec,-0.7393,-90.3518,71.55,86,97,8.01
1,ushuaia,ar,-54.8,-68.3,37.06,75,40,17.27
2,bathsheba,bb,13.2167,-59.5167,77.61,88,75,25.32
3,tasiilaq,gl,65.6145,-37.6368,41.77,83,43,3.2
4,khuzhir,ru,53.1939,107.3397,53.98,80,75,7.61


In [6]:
cities_weather.dtypes

Cities              object
Country             object
Latitude            object
Longitude           object
Temperature (F)     object
Humidity (%)        object
Cloudiness (%)      object
Wind Speed (mph)    object
dtype: object

In [7]:
cities_weather['Latitude'] = pd.to_numeric(cities_weather['Latitude'])
cities_weather['Longitude'] = pd.to_numeric(cities_weather['Longitude'])
cities_weather['Temperature (F)'] = pd.to_numeric(cities_weather['Temperature (F)'])
cities_weather['Humidity (%)'] = pd.to_numeric(cities_weather['Humidity (%)'])
cities_weather['Cloudiness (%)'] = pd.to_numeric(cities_weather['Cloudiness (%)'])
cities_weather['Wind Speed (mph)'] = pd.to_numeric(cities_weather['Wind Speed (mph)'])

In [8]:
import gmaps
gmaps.configure(api_key=g_key)
locations = cities_weather[["Latitude", "Longitude"]].astype(float)
humidity = cities_weather['Humidity (%)'].astype(float)

In [9]:
fig = gmaps.figure(zoom_level = 1, center = (40.1429,-5.1102))
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100, point_radius = 2.4,)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
vacation_spot = cities_weather[(cities_weather['Humidity (%)'] < 80) & (cities_weather['Humidity (%)'] > 70)]
vacation_spot = vacation_spot[vacation_spot['Wind Speed (mph)'] < 10]
vacation_spot = vacation_spot[vacation_spot['Cloudiness (%)'] < 1 ]                             

In [11]:
vacation_spot

,Cities,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
72,rio gallegos,ar,-51.6226,-69.2181,33.85,75,0,6.91
226,petropavlovsk-kamchatskiy,ru,53.0452,158.6483,68.59,73,0,6.71
254,alexandria,eg,31.2156,29.9553,78.82,76,0,9.46
321,kruisfontein,za,-34.0033,24.7314,50.77,71,0,6.15
446,provideniya,ru,64.3833,-173.3000,50.11,76,0,4.47
615,caravelas,br,-17.7125,-39.2481,71.62,79,0,9.26


In [12]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
for index, row in vacation_spot.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    # convert to json
    hotel_name = hotel_name.json()
   # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        vacation_spot.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [13]:
vacation_spot = vacation_spot.reset_index(drop=True)

In [14]:
vacation_spot = (vacation_spot.dropna())
vacation_spot

,Cities,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,rio gallegos,ar,-51.6226,-69.2181,33.85,75,0,6.91,Apart Hotel Austral
1,petropavlovsk-kamchatskiy,ru,53.0452,158.6483,68.59,73,0,6.71,Tri Lyzhi
2,alexandria,eg,31.2156,29.9553,78.82,76,0,9.46,Borg El Arab Beach Hotel
3,kruisfontein,za,-34.0033,24.7314,50.77,71,0,6.15,The Sands @ St. Francis Bay
5,caravelas,br,-17.7125,-39.2481,71.62,79,0,9.26,Lord Pousada & Restaurante


In [15]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in vacation_spot.iterrows()]
location = vacation_spot[['Latitude', 'Longitude']]

In [17]:
marker = gmaps.marker_layer(location, info_box_content =  hotel_info)
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))